In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
Album,table
Artist,table
Customer,table
Employee,table
Genre,table
Invoice,table
InvoiceLine,table
MediaType,table
Playlist,table
PlaylistTrack,table


In [8]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM InvoiceLine il
    INNER JOIN invoice i on il.InvoiceId = i.InvoiceId
    INNER JOIN Customer c on i.CustomerId = c.CustomerId
    WHERE c.Country = "USA"
   )

SELECT
    g.name Genre,
    count(uts.InvoiceLineId) tracks_sold,
    cast(count(uts.InvoiceLineId) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.TrackId = uts.TrackId
INNER JOIN genre g on g.GenreId = t.GenreId
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


Genre,tracks_sold,percentage_sold
Rock,157,0.31781376518218624
Latin,91,0.18421052631578946
Metal,64,0.12955465587044535
Alternative & Punk,50,0.10121457489878542
Jazz,22,0.044534412955465584
Blues,15,0.030364372469635626
TV Shows,14,0.02834008097165992
R&B/Soul,12,0.024291497975708502
Comedy,8,0.016194331983805668
Classical,8,0.016194331983805668


In [16]:
%%sql

WITH customer_support_rep_sales AS
    (
     SELECT
         i.CustomerId,
         c.SupportRepId,
         SUM(i.total) total
     FROM invoice i
     INNER JOIN Customer c ON i.CustomerId = c.CustomerId
     GROUP BY 1,2
    )

SELECT
    e.FirstName || " " || e.LastName employee,
    e.HireDate,
    SUM(csrs.total) total_sales
FROM Customer_support_rep_sales csrs
INNER JOIN employee e ON e.EmployeeId = csrs.SupportRepId
GROUP BY 1;

 * sqlite:///chinook.db
Done.


employee,HireDate,total_sales
Jane Peacock,2002-04-01 00:00:00,833.04
Margaret Park,2003-05-03 00:00:00,775.4
Steve Johnson,2003-10-17 00:00:00,720.16


In [19]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.CustomerId,
       il.*
     FROM InvoiceLine il
     INNER JOIN invoice i ON i.InvoiceId = il.InvoiceId
     INNER JOIN customer c ON c.CustomerId = i.CustomerId
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct CustomerId) customers,
        SUM(UnitPrice) total_sales,
        SUM(UnitPrice) / count(distinct CustomerId) customer_lifetime_value,
        SUM(UnitPrice) / count(distinct InvoiceId) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,523.06,5.747912087912088,40.23538461538461
Canada,8,303.96,5.4278571428571425,37.995
France,5,195.1,5.574285714285714,39.019999999999996
Brazil,5,190.1,5.4314285714285715,38.019999999999996
Germany,4,156.48,5.588571428571428,39.12
United Kingdom,3,112.86,5.3742857142857146,37.62
Czech Republic,2,90.24,6.445714285714286,45.12
Portugal,2,77.24,5.517142857142857,38.62
India,2,75.26,5.78923076923077,37.63
Other,15,604.3,5.755238095238095,40.28666666666666


In [20]:
%%sql

WITH invoice_FirstTrack AS (
  SELECT
    il.InvoiceID AS InvoiceID,
    MIN(il.TrackId) AS FirstTrackId
  FROM
    InvoiceLine il
  GROUP BY
    1
)

-- Use a subquery to select the results of the invoice_first_track CTE and determine whether customers made album purchases
SELECT
  album_purchase,
  COUNT(InvoiceID) AS NumberOfInvoices,
  CAST(COUNT(InvoiceID) AS FLOAT) / (
    SELECT COUNT(*) FROM Invoice
  ) AS Percent
FROM
  (
    SELECT
      ifs.*,
      CASE
        -- Use the EXCEPT operator to compare the tracks in the first invoice with the tracks in subsequent invoices,
        -- and determine whether any tracks from the album were purchased in subsequent invoices.
        -- If the result of the EXCEPT is NULL, it means that all tracks from the album were purchased in subsequent invoices,
        -- and the customer made an album purchase.
        -- If the result of the EXCEPT is not NULL, it means that at least one track from the album was not purchased in subsequent invoices,
        -- and the customer did not make an album purchase.
        WHEN (
          SELECT
            t.TrackId
          FROM
            Track t
          WHERE
            t.AlbumId = (
              SELECT
                t2.AlbumId
              FROM
                Track t2
              WHERE
                t2.TrackId = ifs.FirstTrackId
            )
          EXCEPT
          SELECT
            il2.TrackId
          FROM
            InvoiceLine il2
          WHERE
            il2.InvoiceID = ifs.InvoiceID
        ) IS NULL
        AND (
          SELECT
            il2.TrackId
          FROM
            InvoiceLine il2
          WHERE
            il2.InvoiceID = ifs.InvoiceID
          EXCEPT
          SELECT
            t.TrackId
          FROM
            Track t
          WHERE
            t.AlbumId = (
              SELECT
                t2.AlbumId
              FROM
                Track t2
              WHERE
                t2.TrackId = ifs.FirstTrackId
            )
        ) IS NULL
        THEN "yes"
        ELSE "no"
      END AS album_purchase
    FROM
      invoice_FirstTrack ifs
  ) subquery
-- Group by album_purchase to get the counts and percentages for each type of purchase
GROUP BY
  album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,NumberOfInvoices,Percent
no,410,0.9951456310679612
yes,2,0.0048543689320388345
